# Develop Model

In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import numpy as np
from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [14]:
def evauation_model(pred, y_val):
  score_MSE = round(mean_squared_error(pred, y_val),2)
  score_MAE = round(mean_absolute_error(pred, y_val),2)
  score_r2score = round(r2_score(pred, y_val),2)
  return score_MSE, score_MAE, score_r2score


def models_score(model_name, train_data, y_train, val_data, y_val):
    model_list = ["Decision_Tree", "Random_Forest", "XGboost_Regressor"]
    # model_1
    if model_name == "Decision_Tree":
        reg = DecisionTreeRegressor(random_state=42)
    # model_2
    elif model_name == "Random_Forest":
        reg = RandomForestRegressor(random_state=42)

    # model_3
    elif model_name == "XGboost_Regressor":
        reg = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, )
    else:
        print("please enter correct regressor name")

    if model_name in model_list:
        reg.fit(train_data, y_train)
        pred = reg.predict(val_data)

        score_MSE, score_MAE, score_r2score = evauation_model(pred, y_val)
        return round(score_MSE, 2), round(score_MAE, 2), round(score_r2score, 2)

In [15]:
def evauation_model(pred, y_val):
  score_MSE = round(mean_squared_error(pred, y_val), 2)
  score_MAE = round(mean_absolute_error(pred, y_val), 2)
  score_r2score = round(r2_score(pred, y_val), 2)
  return score_MSE, score_MAE, score_r2score


In [16]:
def hyperparameter_tuning(space):
    model = xgb.XGBRegressor(n_estimators=space['n_estimators'], max_depth=int(space['max_depth']),
                             gamma=space['gamma'],
                             reg_alpha=int(space['reg_alpha']), min_child_weight=space['min_child_weight'],
                             colsample_bytree=space['colsample_bytree'], objective="reg:squarederror")

    score_cv = cross_val_score(
        model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error").mean()
    return {'loss': -score_cv, 'status': STATUS_OK, 'model': model}

In [19]:
data = pd.read_csv("Fish.csv")
data_cleaned = data.drop("Weight", axis=1)
y = data['Weight']
#%%
x_train, x_test, y_train, y_test = train_test_split(data_cleaned,y, test_size=0.2, random_state=42)
label_encoder = LabelEncoder()
x_train['Species'] = label_encoder.fit_transform(x_train['Species'].values)
x_test['Species'] = label_encoder.transform(x_test['Species'].values)
model_list = ["Decision_Tree","Random_Forest","XGboost_Regressor"]
#%%
result_scores = []
for model in model_list:
    score = models_score(model, x_train, y_train, x_test, y_test)
    result_scores.append((model, score[0], score[1],score[2]))
    print(model,score)

df_result_scores = pd.DataFrame(result_scores,columns=["model","mse","mae","r2score"])
df_result_scores
#%%
num_estimator = [100, 150, 200, 250]

space = {'max_depth': hp.quniform("max_depth", 3, 18, 1),
         'gamma': hp.uniform('gamma', 1, 9),
         'reg_alpha': hp.quniform('reg_alpha', 30, 180, 1),
         'reg_lambda': hp.uniform('reg_lambda', 0, 1),
         'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
         'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
         'n_estimators': hp.choice("n_estimators", num_estimator),}

Decision_Tree (7005.89, 48.71, 0.95)
Random_Forest (4659.44, 44.66, 0.97)
XGboost_Regressor (5151.54, 41.05, 0.96)


In [21]:
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

2023-02-03 23:07:39.123 build_posterior_wrapper took 0.005979 seconds
2023-02-03 23:07:39.124 TPE using 0 trials


  1%|          | 1/100 [00:00<01:38,  1.00trial/s, best loss: 46.201318116995]

2023-02-03 23:07:40.124 build_posterior_wrapper took 0.005984 seconds
2023-02-03 23:07:40.125 TPE using 1/1 trials with best loss 46.201318


  2%|▏         | 2/100 [00:01<01:15,  1.31trial/s, best loss: 46.201318116995]

...

 98%|█████████▊| 98/100 [01:55<00:02,  1.35s/trial, best loss: 38.42314930337759]

2023-02-03 23:09:35.040 build_posterior_wrapper took 0.005984 seconds
2023-02-03 23:09:35.042 TPE using 98/98 trials with best loss 38.423149


 99%|█████████▉| 99/100 [01:56<00:01,  1.27s/trial, best loss: 38.42314930337759]

2023-02-03 23:09:36.112 build_posterior_wrapper took 0.004987 seconds
2023-02-03 23:09:36.116 TPE using 99/99 trials with best loss 38.423149


100%|██████████| 100/100 [01:58<00:00,  1.18s/trial, best loss: 38.42314930337759]


In [ ]:
print(best)

In [ ]:
#%%
best['max_depth'] = int(best['max_depth'])  # convert to int
# assing n_estimator because it returs the index
best["n_estimators"] = num_estimator[best["n_estimators"]]
best_xgboost_model = xgb.XGBRegressor(**best)
best_xgboost_model.fit(x_train, y_train)
pred = best_xgboost_model.predict(x_test)
score_MSE, score_MAE, score_r2score = evauation_model(pred, y_test)
to_append = ["XGboost_hyper_tuned", score_MSE, score_MAE, score_r2score]
df_result_scores.loc[len(df_result_scores)] = to_append

best_xgboost_model.save_model("best_model.json")


# Prediction

In [7]:
data = pd.read_csv("Fish.csv")
data_cleaned = data.drop("Weight", axis=1)
y = data['Weight']
x_train, x_test, y_train, y_test = train_test_split(
    data_cleaned, y, test_size=0.2, random_state=42)
label_encoder = LabelEncoder()
x_train['Species'] = label_encoder.fit_transform(x_train['Species'].values)
x_test['Species'] = label_encoder.transform(x_test['Species'].values)
#save label encoder classes
np.save('classes.npy', label_encoder.classes_)


In [8]:
# load model
best_xgboost_model = xgb.XGBRegressor()
best_xgboost_model.load_model("best_model.json")
pred = best_xgboost_model.predict(x_test)
score_MSE, score_MAE, score_r2score = evauation_model(pred, y_test)
print(score_MSE, score_MAE, score_r2score)
#%%


3478.97 40.98 0.97


In [9]:
loaded_encoder = LabelEncoder()
loaded_encoder.classes_ = np.load('classes.npy',allow_pickle=True)
print(x_test.shape)
input_species = loaded_encoder.transform(np.expand_dims("Parkki",-1))
print(int(input_species))
inputs = np.expand_dims([int(input_species),15,20,10,4,5],0)
print(inputs.shape)
prediction = best_xgboost_model.predict(inputs)
print("final pred", np.squeeze(prediction,-1))

(32, 6)
1
(1, 6)
final pred 72.352615
